In [0]:
from pyspark.sql.functions import *

In [0]:
df_customers = spark.read.format("delta")\
        .load("/Volumes/adventure_work/bronze/data/Customers/")

In [0]:
df_customers.display()

In [0]:
df_customers = df_customers.withColumn("FullName", concat(df_customers.FirstName, lit(" "), df_customers.LastName))

In [0]:
df_customers = df_customers.na.drop(subset=["Gender", "Prefix"])

In [0]:
df_customers.display()

In [0]:
df_customers = df_customers.withColumn("unix_date",from_unixtime(unix_timestamp('BirthDate', 'M/d/yyy')))

In [0]:
df_customers_cleaned = df_customers.withColumn("BirthDate", to_date("unix_date")).drop("unix_date")

In [0]:
df_customers_cleaned = df_customers_cleaned.withColumn(
        "AnnualIncome", regexp_replace(col("AnnualIncome"), "[$,]", "").cast("int")
    )\
    .withColumn("TotalChildren", col("TotalChildren").cast("int"))\
    .withColumn("CustomerKey", col("CustomerKey").cast("int"))\
    .withColumn("FirstName", initcap("FirstName"))\
    .withColumn("LastName", initcap("LastName"))\
    .withColumn("Prefix", initcap("Prefix"))\
    .withColumn("FullName", initcap("FullName"))\
    .drop("ingestion_timestamp")\
    .withColumn("processed_date", current_date())

In [0]:
df_customers_cleaned.display()

In [0]:
df_customers_cleaned.write.mode("append")\
    .format("delta")\
    .save("/Volumes/adventure_work/silver/data/Customers/")